In [3]:
import hashlib
import hashlib
import json
import torch
import random

from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    get_cosine_schedule_with_warmup
)
from torch.optim import AdamW
class Config:
    train_path = './DuReaderQG/train.json'
    valid_path = './DuReaderQG/dev.json'
    model_path = './models/langboat/mengzi-t5-base'
    save_dir = './best_model'
    max_source_length = 1024
    max_target_length = 128
    batch_size = 4
    accum_steps = 4
    epochs = 30
    val_samples_per_epoch = 1
    seed = 42
    valid_shuffle = True  


In [4]:
torch.manual_seed(Config.seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# 数据加载
import json
def load_train_data(path):
    # 加载训练数据 
    with open(path,'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f if line.strip()]
train_data = load_train_data(Config.train_path)
print(len(train_data))

14520


In [6]:
import hashlib
def load_valid_data(path):
    """加载验证数据（合并相同context+question）"""
    grouped = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                sample = json.loads(line)
                key = hashlib.md5(
                    (sample["context"] + sample["question"]).encode()
                ).hexdigest()
                if key not in grouped:
                    grouped[key] = {
                        "context": sample["context"],
                        "question": sample["question"],
                        "answers": [],
                        "ids": []
                    }
                grouped[key]["answers"].append(sample["answer"])
                grouped[key]["ids"].append(sample["id"])
    return list(grouped.values())


valid_data = load_valid_data(Config.valid_path)


In [6]:
print(len(valid_data))

700


In [7]:
print(valid_data[:2])

[{'context': '年基准利率4.35%。 从实际看,贷款的基本条件是: 一是中国大陆居民,年龄在60岁以下; 二是有稳定的住址和工作或经营地点; 三是有稳定的收入来源; 四是无不良信用记录,贷款用途不能作为炒股,赌博等行为; 五是具有完全民事行为能力。', 'question': '2017年银行贷款基准利率', 'answers': ['年基准利率4.35%', '4.35%'], 'ids': [0, 1]}, {'context': 'U系列是最好的，采用国际顶尖技术（由格力自主研发）双级变频压缩机，提高压缩机运转效率，制冷制热能力更强劲；1赫兹变频技术，使空调相当于一个15 W电灯泡，更加节能省电；送风面积广，风力大；生态风，净化空气。非常不错，现在国美在做活动，可以了解一下。', 'question': '格力空调哪个系列好', 'answers': ['U系列'], 'ids': [2]}]


In [7]:
class QADataset:
    def __init__(self,data,is_train=True):
        self.data = data
        self.is_train = is_train
    
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        if self.is_train:
            return {
                'context': item['context'],
                'question': item['question'],
                'answer': item['answer'],
                "id" : item['id']
            }
        else:
            return {
                'context': item['context'],
                'question': item['question'],
                'answers': item['answers'],  # 在验证集上，答案是一个列表
                'id': item['ids'][0]
            }

In [14]:
train_data = QADataset(train_data, is_train=True)
valid_data = QADataset(valid_data, is_train=False)

In [9]:
train_data[1]

{'context': '选择燃气热水器时，一定要关注这几个问题：1、出水稳定性要好，不能出现忽热忽冷的现象2、快速到达设定的需求水温3、操作要智能、方便4、安全性要好，要装有安全报警装置 市场上燃气热水器品牌众多，购买时还需多加对比和仔细鉴别。方太今年主打的磁化恒温热水器在使用体验方面做了全面升级：9秒速热，可快速进入洗浴模式；水温持久稳定，不会出现忽热忽冷的现象，并通过水量伺服技术将出水温度精确控制在±0.5℃，可满足家里宝贝敏感肌肤洗护需求；配备CO和CH4双气体报警装置更安全（市场上一般多为CO单气体报警）。另外，这款热水器还有智能WIFI互联功能，只需下载个手机APP即可用手机远程操作热水器，实现精准调节水温，满足家人多样化的洗浴需求。当然方太的磁化恒温系列主要的是增加磁化功能，可以有效吸附水中的铁锈、铁屑等微小杂质，防止细菌滋生，使沐浴水质更洁净，长期使用磁化水沐浴更利于身体健康。',
 'question': '燃气热水器哪个牌子好',
 'answer': '方太',
 'id': 1}

In [10]:
valid_data[5]

{'context': '密歇根州。地理位置：密歇根州（Michigan）位于美国最北部，由两大半岛组成，分隔两半岛的水面叫做麦基诺水道。南为下半岛，是该州的主体，面积较大，其南境西半部接印第安纳州，东半部接俄亥俄州。半岛西、北、东三方面均为湖泊，西、北为密歇根湖，东北为休伦湖，东为圣克莱尔湖与圣克莱尔河，东南是伊利湖。北部为上半岛，面积比下半岛小，北滨苏必利尔湖，南临密歇根湖，西南邻威斯康星州，东端为圣马里斯河及苏运河。气候与面积：州面积250,493平方公里，居美国50州第11位。位于五大湖区，受湖风调剂，气候温和。北方的苏圣玛丽城平均最高温度为10℃，平均最低温度为-1℃。东南部的底特律市平均最高温度为14℃，平均最低温度为6℃。上半岛的生长期约为3个月，而下半岛的南部地区长达6个月。年平均降水量838毫米，南部较多，达914毫米。城市与人口：人口987.6万（2011年），居全美第8位，白人占总人口的79.6%，黑人占14%。密州有83个郡，州府兰辛市（Lansing），人口11.6万，为州政治、文化及教育中心。底特律（Detroit）是密州最大城市，人口87万。大底特律地区华侨华人较为集中。历史：17世纪时为印第安人居住地，1668年法国殖民者建立第一个定居点，1701年底特律成为皮毛贸易中心，1783年以后归属美国。1837年加入联邦，成为美国第26个州。',
 'question': 'mi是美国哪个州的缩写',
 'answers': ['密歇根州', '密歇根州（Michigan）', 'Michigan'],
 'id': 8}

In [18]:
# 定义验证集分块器，在实际开发中由于验证集通常较小，分块器的作用是将验证集划分为多个小块，
# 以便在每个epoch中使用不同的验证样本进行评估。

import torch
class ValidChunk():
    def __init__(self,dataset,epochs):
        self.dataset = dataset
        self.total_samples = len(dataset)
        self.total_epochs = epochs
        self.samples_per_epoch = self.total_samples // epochs
        self.indices = torch.arange(0, self.total_samples).tolist()
        
        
        if self.total_samples % epochs != 0:
            pad = self.total_epochs - (self.total_samples % epochs)
            self.indices += self.indices[:pad]
            self.total_samples += pad
            
            
        if Config.valid_shuffle:
            random.shuffle(self.indices)
    def get_chunk_indices(self, epoch):
        start = epoch * self.samples_per_epoch
        end = start + self.samples_per_epoch
        return self.indices[start:end]
valid_chunker = ValidChunk(valid_data, Config.epochs) # 后期实际上没有调用,
tokenizer = AutoTokenizer.from_pretrained(Config.model_path) # 分词器

def train_collate(batch):
    """收集每一个batch的训练数据"""
    inputs = [f"question:{item['question']} context:{item['context']}" for item in batch]
    targets = [item['answer'] for item in batch]
    
    source = tokenizer(inputs, 
                        max_length=Config.max_source_length, 
                        padding='max_length', 
                        truncation=True, 
                        return_tensors='pt')
    target = tokenizer(targets,
                        max_length=Config.max_target_length, 
                        padding='max_length', 
                        truncation=True, 
                        return_tensors='pt').input_ids
    
    target[target == tokenizer.pad_token_id] = -100  # 忽略填充部分
    return {
        'input_ids': source.input_ids,
        'attention_mask': source.attention_mask,
        'labels': target
    }
def valid_collates(batch):
    """收集每一个batch的验证数据"""
    inputs = [f"question:{item['question']} context:{item['context']}" for item in batch]
    processed = tokenizer(
        inputs,
        max_length=Config.max_source_length,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    return {
        'input_ids': processed.input_ids,
        'attention_mask': processed.attention_mask,
        'ids': [item['id'] for item in batch],  # 保留ID以便后续使用
        'context': [item['context'] for item in batch],
        'question': [item['question'] for item in batch],
        'answers': [item['answers'] for item in batch]  # 保留答案
    }





In [13]:
model = T5ForConditionalGeneration.from_pretrained(Config.model_path).to(device)

In [15]:
#定义优化器
optimizer = AdamW(model.parameters(), lr=1e-4)

In [19]:
# dataloader
train_loader = DataLoader(
    train_data,
    batch_size=Config.batch_size,
    shuffle=True,
    collate_fn=train_collate
)


In [20]:
# 整个数据集在每一个epoch中都被分成了多个小批次，len(train_loader) 是mini-batch的数量
num_training_steps = len(train_loader) * Config.epochs // Config.accum_steps
# 预热步数
num_warmup_steps = int(0.1 * num_training_steps)  # 10% warmup，可调
# 定义学习率调度器
schecduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [22]:

from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu


class BleuEvaluator:
    def __init__(self):
        self.smooth = SmoothingFunction().method1
        self.weights = {
            1: (1, 0, 0, 0),
            2: (0.5, 0.5, 0, 0),
            3: (1/3, 1/3, 1/3, 0),
            4: (0.25, 0.25, 0.25, 0.25)
        }

    def calc_bleu(self, pred, refs):
        pred_tokens = list(pred.strip())
        ref_tokens = [list(r.strip()) for r in refs]
        return {
            f"BLEU-{n}": sentence_bleu(
                ref_tokens, pred_tokens,
                weights=self.weights[n],
                smoothing_function=self.smooth
            ) for n in range(1, 5)
        }

    @staticmethod
    def dynamic_score(scores, pred_len):
        if pred_len <= 2:
            return scores["BLEU-1"] * 0.6 + scores["BLEU-2"] * 0.4
        elif 3 <= pred_len <= 5:
            return scores["BLEU-2"] * 0.5 + scores["BLEU-3"] * 0.3 + scores["BLEU-4"] * 0.2
        else:
            return scores["BLEU-4"] * 0.7 + scores["BLEU-3"] * 0.3

In [23]:
# 假设你已经定义好 train_dataloader
batch = next(iter(train_loader))  # train_loader 就是你的 DataLoader 对象


In [24]:

# 将张量移动到 CPU，便于查看
for key, value in batch.items():
    print(f"{key}:")
    print(value if isinstance(value, list) else value.shape)
    print("-" * 50)

input_ids:
torch.Size([4, 1024])
--------------------------------------------------
attention_mask:
torch.Size([4, 1024])
--------------------------------------------------
labels:
torch.Size([4, 128])
--------------------------------------------------


In [25]:
print("input示例")
print(batch["input_ids"][0]) # 第一个样本'input_ids'

input示例
tensor([   7, 3454, 2055,  ...,    0,    0,    0])


In [26]:
print("解码为文本:")
print(tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True))

解码为文本:
question:人力资源管理专业属于什么类 context:人力资源管理专业学科:管理学 (Management)门类:工商管理类(Mater of Business Administration)专业名称:人力资源管理 (Human Resources Management)业务培养目标:本专业培养具备管理、经济、法律及人力资源管理等方面的知识和能力,能在事业单位及政府部门从事人力资源管理以及教学、科研方面工作的工商管理学科高级专门人才。业务培养要求:本专业学生上要学习管理学、经济学及人力资源管理方面的基本理论和基本知识,受到人力资源管理方法与技巧方面的基本训练,具有分析和解决人力资源管理问题的基本能力。毕业生应获得以下几方面的知识和能力:1.掌握管理学、经济学及人力资源管理的基本理论、基本知识;2.掌握人力资源管理的定性、定量分析方法;3.具有较强的语言与文字表达、人际沟通、组织协调及领导的基本能力;4.熟悉与人力资源管理有关的方针、政策及法规;5.了解本学科理论前沿与发展动态;6.掌握文献检索、资料查询的基本方法,具有一定科学研究和实际工作能力。主干学科:经济学、工商管理主要课程:管理学、微观经济学、宏观经济学、管理信息系统,统计学、会计学、财务管理、市场营销、经济法、人力资源管理、组织行为学、劳动经济学。 主要实践性教学环节:包括课程实习与毕业实习,一般安排10-12周。修业年限:四年授予学位:管理学学士(荣誉生)


In [29]:
print("\nlabels 示例:")
print(batch["labels"][0])
print("解码为文本:")
label_ids = batch["labels"][0]
# 替换掉 -100（忽略值）为 pad_token_id，再解码
label_ids = [id if id != -100 else tokenizer.pad_token_id for id in label_ids]
print(tokenizer.decode(label_ids, skip_special_tokens=True))


labels 示例:
tensor([    7, 20829,   373,     1,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -

In [ ]:
import numpy as np

best_dynamic = 0.0
loss_history = []
lr_history = []
bleu1_history = []
bleu2_history = []
bleu3_history = []
bleu4_history = []
dynamic_history = []

def validate(loader):
    evaluator = BleuEvaluator()
    all_bleu = []
    all_dynamic = []
    displayed = set()

    for batch in loader:
        inputs = batch["input_ids"].to(device)
        generated = model.generate(
            inputs,
            max_length=Config.max_target_length,
            num_beams=5,
            early_stopping=True
        )
        preds = tokenizer.batch_decode(generated, skip_special_tokens=True)

        for i, (pred, refs) in enumerate(zip(preds, batch["answers"])):
            bleu = evaluator.calc_bleu(pred, refs)
            all_bleu.append(bleu)

            pred_len = len(list(pred.strip()))
            dynamic = BleuEvaluator.dynamic_score(bleu, pred_len)
            all_dynamic.append(dynamic)

    avg_bleu = {
        f"BLEU-{n}": np.mean([b[f"BLEU-{n}"] for b in all_bleu]) * 100
        for n in range(1, 5)
    }
    avg_dynamic = np.mean(all_dynamic) * 100
    return avg_bleu, avg_dynamic

In [ ]:
def save_model(epoch, dynamic_score):
    save_path = Path(Config.save_dir) / f"epoch_{epoch}_dynamic_{dynamic_score:.2f}"
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"\n模型保存至: {save_path}")
